In [4]:
import os
import torch
from copy import deepcopy
import numpy as np
import xarray as xr
import pandas as pd
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import netCDF4 as nc

In [5]:
train2 = xr.open_dataset('../data/SODA_train.nc')
label2 = xr.open_dataset('../data/SODA_label.nc')

train_sst2 = train2['sst'][:, :12].values  # (100, 12, 24, 72)
train_t3002 = train2['t300'][:, :12].values
train_ua2 = train2['ua'][:, :12].values
train_va2 = train2['va'][:, :12].values
train_label2 = label2['nino'][:, 12:36].values

In [6]:
train2

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, month: 36, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
Data variables:
    sst      (year, month, lat, lon) float32 ...
    t300     (year, month, lat, lon) float32 ...
    ua       (year, month, lat, lon) float64 ...
    va       (year, month, lat, lon) float64 ...

In [52]:
def data_enhance(df, data_type='feature'):
    if data_type=='feature':
        df_ = df
        for i in range(1, 12):
            dfi = df_.reshape(-1, 24, 72)
            dfi = dfi[i:-(12-i), :, :]
            dfi = dfi.reshape(-1, 12, 24, 72)
            df = np.concatenate((df, dfi), axis=0)
    elif data_type=='label':
        df_ = np.concatenate((df[:, :12], df[-1:, 12:24], df[-1:, 24:36]), axis=0)
        for i in range(1, 12):
            df_ = df_.reshape(-1)
            dfi1 = df_[i:-(36-i)].reshape(-1, 12)
            dfi2 = df_[i+12:-(24-i)].reshape(-1, 12)
            dfi3 = df_[i+24:-(12-i)].reshape(-1, 12)
            dfi = np.concatenate((dfi1, dfi2, dfi3), axis=1)
            df = np.concatenate((df, dfi), axis=0)
            
    return df

# train_sst2.reshape(-1, 24, 72)
tmp = data_enhance(train_sst2)

In [53]:
# label2_ = xr.Dataset.to_dataframe(label2['nino'])
label2_ = label2['nino'].values
label2_ = data_enhance(label2_, data_type='label')

In [54]:
print(tmp.shape)
print(label2_.shape)

(1189, 12, 24, 72)
(1189, 36)


In [35]:
label2_ = np.array(label2_['nino'])
label2_ = 

(3600,)

In [26]:
np.array(label2['nino']).shape

(100, 36)

In [27]:
label2['nino'].values.shape

(100, 36)